<a href="https://colab.research.google.com/github/sean-halpin/guided_diffusion/blob/main/conditional_celeba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
continue_training = True

In [3]:
import os, os.path

first_run = True
if os.path.exists('/content/data/celeba/img_align_celeba') and len(os.listdir('/content/data/celeba/img_align_celeba')) == 202599:
  first_run = False

In [4]:
if first_run:
  !rm -rf /content/images/*
  !mkdir -p /content/images/

In [5]:
from typing import Dict, Tuple
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# from torchvision.datasets import MNIST
from torchvision import transforms
from torchvision.utils import save_image, make_grid

In [ ]:
!python -m pip uninstall matplotlib -y
!pip install matplotlib==3.1.3

### UNet

In [7]:
%%capture
!sudo apt -qq install git-lfs
!git config --global credential.helper store

In [ ]:
!pip install diffusers[training]==0.1.3

In [9]:
from diffusers import UNet2DConditionModel

### Dataset

In [10]:
# from torchvision.datasets import CIFAR10
from torchvision.datasets import CelebA

In [11]:
x_size = 32
y_size = x_size

In [12]:
tf = transforms.Compose(
  [
    transforms.Resize((x_size, y_size)),
    transforms.ToTensor(), 
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ]
)

In [13]:
if first_run:
  from google.colab import drive
  drive.mount('/content/drive/')
  !mkdir -p ./data
  !rm -rf ./data/*
  !cp -R ./drive/MyDrive/celeba/* ./data/celeba
  !unzip "./data/celeba/img_align_celeba.zip" -d "./data/celeba"

In [14]:
import time 

dataset = None
result = None
while result is None:
    try:
      # connect
      dataset = CelebA(
        "./data/",
        download=False,
        transform=tf,
      )
      result = True
    except Exception as e:
      print(e)
      # break
      time.sleep(60)
      pass


In [15]:
subset = torch.utils.data.Subset(dataset, range(0,320))

In [16]:
batch_size_ = 16
eval_batch_size_ = 1

In [17]:
# dataloader = DataLoader(subset, batch_size=32, shuffle=True, num_workers=2)
dataloader = DataLoader(dataset, batch_size=batch_size_, shuffle=True, num_workers=2)

### Denoising Diffusion Probabilistic Model

In [18]:
!pip install Accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import torch

from tqdm.auto import tqdm

from diffusers import DiffusionPipeline
from diffusers import DDPMScheduler, UNet2DConditionModel


class OldDDPMConditionalPipeline(DiffusionPipeline):
    def __init__(self, unet: UNet2DConditionModel, scheduler):
        super().__init__()
        scheduler = scheduler.set_format("pt")
        self.register_modules(unet=unet, scheduler=scheduler)

    @torch.no_grad()
    def __call__(self, batch_size=1, generator=None, torch_device=None, output_type="pil", hidden_states=None):
        if torch_device is None:
            torch_device = "cuda" if torch.cuda.is_available() else "cpu"

        self.unet.to(torch_device)

        # Sample gaussian noise to begin loop
        image = torch.randn(
            (batch_size, self.unet.in_channels, self.unet.sample_size, self.unet.sample_size),
            generator=generator,
        )
        image = image.to(torch_device)

        # set step values
        self.scheduler.set_timesteps(1000)

        for t in tqdm(self.scheduler.timesteps):
            # 1. predict noise model_output
            model_output = self.unet(image, t, hidden_states)["sample"]

            # 2. compute previous image: x_t -> t_t-1
            image = self.scheduler.step(model_output, t, image)["prev_sample"]

        image = (image / 2 + 0.5).clamp(0, 1)
        image = image.cpu().permute(0, 2, 3, 1).numpy()
        if output_type == "pil":
            image = self.numpy_to_pil(image)

        return {"sample": image}

In [20]:
torch.cuda.is_available()

True

In [21]:
import torch

from tqdm.auto import tqdm

from diffusers import DiffusionPipeline
from diffusers import DDPMScheduler, UNet2DConditionModel

emb = nn.Embedding(2, 40, padding_idx=0)
lin = nn.Linear(40, 1280)

class DDPMConditionalPipeline(DiffusionPipeline):
    def __init__(self, unet: UNet2DConditionModel, scheduler):
        super().__init__()
        scheduler = scheduler.set_format("pt")
        self.register_modules(unet=unet, scheduler=scheduler)

    @torch.no_grad()
    def __call__(self, guidance_scale=10.0, batch_size=1, generator=None, torch_device=None, output_type="pil", hidden_states=None):
        if torch_device is None:
            torch_device = "cuda" if torch.cuda.is_available() else "cpu"

        self.unet.to(torch_device)

        # Sample gaussian noise to begin loop
        latents = torch.randn(
            (batch_size, self.unet.in_channels, self.unet.sample_size, self.unet.sample_size),
            generator=generator,
        )
        image = latents.to(torch_device)

        # set step values
        self.scheduler.set_timesteps(1000)

        for t in tqdm(self.scheduler.timesteps):
            # 1. predict noise noise_pred
            # latents_input = torch.cat([latents] * 2)
            no_condition_attrs = torch.tensor([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]).long()
            y = lin(emb(no_condition_attrs)).to(torch_device)
            # context = torch.cat([[y], [hidden_states]])
            noise_prediction_text = self.unet(image, t, hidden_states)["sample"]
            noise_pred_uncond = self.unet(image, t, y)["sample"]

            # noise_pred_uncond, noise_prediction_text = noise_pred.chunk(2)
            noise_pred = noise_pred_uncond + guidance_scale * (noise_prediction_text - noise_pred_uncond)


            # 2. compute previous image: x_t -> t_t-1
            image = self.scheduler.step(noise_pred, t, image)["prev_sample"]

        image = (image / 2 + 0.5).clamp(0, 1)
        image = image.cpu().permute(0, 2, 3, 1).numpy()
        if output_type == "pil":
            image = self.numpy_to_pil(image)

        return {"sample": image}

In [22]:
import argparse
import os

import torch
import torch.nn.functional as F

from accelerate import Accelerator
from accelerate.logging import get_logger
# from datasets import load_dataset
# from diffusers import DDPMPipeline, DDPMScheduler, UNet2DConditionModel
from diffusers import DDPMScheduler, UNet2DConditionModel
from diffusers.hub_utils import init_git_repo, push_to_hub
from diffusers.optimization import get_scheduler
from diffusers.training_utils import EMAModel
from torchvision.transforms import (
    CenterCrop,
    Compose,
    InterpolationMode,
    Normalize,
    RandomHorizontalFlip,
    Resize,
    ToTensor,
)
from tqdm.auto import tqdm

In [23]:
!mkdir -p logs

In [24]:
parser = argparse.ArgumentParser(description="Simple example of a training script.")
parser.add_argument('-f')
parser.add_argument("--local_rank", type=int, default=-1)
parser.add_argument("--dataset", type=str, default="CelebA")
parser.add_argument("--dataset_name", type=str, default="CelebA")
parser.add_argument("--dataset_config_name", type=str, default=None)
parser.add_argument("--train_data_dir", type=str, default=None, help="A folder containing the training data.")
parser.add_argument("--output_dir", type=str, default="diffusion_conditional")
parser.add_argument("--overwrite_output_dir", action="store_true")
parser.add_argument("--cache_dir", type=str, default=None)
parser.add_argument("--resolution", type=int, default=x_size)
parser.add_argument("--train_batch_size", type=int, default=batch_size_)
parser.add_argument("--eval_batch_size", type=int, default=eval_batch_size_)
parser.add_argument("--num_epochs", type=int, default=100)
parser.add_argument("--save_images_epochs", type=int, default=1)
parser.add_argument("--save_model_epochs", type=int, default=1)
parser.add_argument("--gradient_accumulation_steps", type=int, default=1)
parser.add_argument("--learning_rate", type=float, default=1e-4)
parser.add_argument("--lr_scheduler", type=str, default="cosine")
parser.add_argument("--lr_warmup_steps", type=int, default=500)
parser.add_argument("--adam_beta1", type=float, default=0.95)
parser.add_argument("--adam_beta2", type=float, default=0.999)
parser.add_argument("--adam_weight_decay", type=float, default=1e-6)
parser.add_argument("--adam_epsilon", type=float, default=1e-08)
parser.add_argument("--use_ema", action="store_true", default=True)
parser.add_argument("--ema_inv_gamma", type=float, default=1.0)
parser.add_argument("--ema_power", type=float, default=3 / 4)
parser.add_argument("--ema_max_decay", type=float, default=0.9999)
parser.add_argument("--push_to_hub", default=True)
parser.add_argument("--use_auth_token", action="store_true")
parser.add_argument("--hub_token", type=str, default=None)
parser.add_argument("--hub_model_id", type=str, default="diffusion_conditional")
parser.add_argument("--hub_private_repo", default=False)
parser.add_argument("--logging_dir", type=str, default="logs")
parser.add_argument(
    "--mixed_precision",
    type=str,
    default="fp16",
    choices=["no", "fp16", "bf16"],
    help=(
        "Whether to use mixed precision. Choose"
        "between fp16 and bf16 (bfloat16). Bf16 requires PyTorch >= 1.10."
        "and an Nvidia Ampere GPU."
    ),
)

args = parser.parse_args()

In [25]:
import numpy as np
import matplotlib.pyplot as plt

In [26]:
logging_dir = os.path.join('.', '/logs')
accelerator = Accelerator(
    mixed_precision=args.mixed_precision,
    log_with="tensorboard",
    logging_dir=logging_dir,
)

In [27]:
model = UNet2DConditionModel(
    sample_size=args.resolution,
    in_channels=3,
    out_channels=3,
    layers_per_block=2,
    block_out_channels=(128, 128, 256, 256),
    # block_out_channels=(128, 128, 256, 256, 512, 512),
    down_block_types=( 
        "CrossAttnDownBlock2D", 
        "CrossAttnDownBlock2D", 
        "CrossAttnDownBlock2D", 
        "DownBlock2D"
    ), 
    up_block_types=(
        "UpBlock2D", 
        "CrossAttnUpBlock2D", 
        "CrossAttnUpBlock2D", 
        "CrossAttnUpBlock2D"
      ),
)

noise_scheduler = DDPMScheduler(num_train_timesteps=1000, tensor_format="pt")

In [28]:
from diffusers import DiffusionPipeline
if continue_training:
  previous_model = \
    DDPMConditionalPipeline.from_pretrained('shalpin87/diffusion_conditional')
  model = previous_model.unet
  noise_scheduler = previous_model.scheduler

In [29]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=args.learning_rate,
    betas=(args.adam_beta1, args.adam_beta2),
    weight_decay=args.adam_weight_decay,
    eps=args.adam_epsilon,
)

train_dataloader = dataloader
lr_scheduler = get_scheduler(
    args.lr_scheduler,
    optimizer=optimizer,
    num_warmup_steps=args.lr_warmup_steps,
    num_training_steps=(len(train_dataloader) * args.num_epochs) // args.gradient_accumulation_steps,
)

model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, lr_scheduler
)

ema_model = EMAModel(model, inv_gamma=args.ema_inv_gamma, power=args.ema_power, max_value=args.ema_max_decay)

In [30]:
if args.push_to_hub:
    repo = init_git_repo(args, at_init=True)
    repo.git_pull()

/content/diffusion_conditional is already a clone of https://huggingface.co/shalpin87/diffusion_conditional. Make sure you pull the latest changes with `repo.git_pull()`.


In [31]:
if accelerator.is_main_process:
    run = os.path.split('.')[-1].split(".")[0]
    accelerator.init_trackers(run)

In [32]:
for batch, y in (train_dataloader):
  print(batch.shape)
  print(y.shape)
  break

torch.Size([16, 3, 32, 32])
torch.Size([16, 40])


In [ ]:
global_step = 0
emb = nn.Embedding(2, 40, padding_idx=0)
lin = nn.Linear(40, 1280)
for epoch in range(args.num_epochs):
    model.train()
    progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
    progress_bar.set_description(f"Epoch {epoch}")
    # for step, batch in enumerate(train_dataloader):
    for batch, batch_y in (train_dataloader):
        # clean_images = batch["input"]
        clean_images = batch #["input"]
        # Sample noise that we'll add to the images
        noise = torch.randn(clean_images.shape).to(clean_images.device)
        bsz = clean_images.shape[0]
        # Sample a random timestep for each image
        timesteps = torch.randint(
            0, noise_scheduler.num_train_timesteps, (bsz,), device=clean_images.device
        ).long()

        # Add noise to the clean images according to the noise magnitude at each timestep
        # (this is the forward diffusion process)
        noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

        with accelerator.accumulate(model):
            # Predict the noise residual          
            y = (lin(emb(batch_y.to("cpu")))).to("cuda")
            noise_pred = model(noisy_images, timesteps, y)["sample"]
            loss = F.mse_loss(noise_pred, noise)
            accelerator.backward(loss)

            accelerator.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            lr_scheduler.step()
            if args.use_ema:
                ema_model.step(model)
            optimizer.zero_grad()

        progress_bar.update(1)
        logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
        if args.use_ema:
            logs["ema_decay"] = ema_model.decay
        progress_bar.set_postfix(**logs)
        accelerator.log(logs, step=global_step)
        global_step += 1
    progress_bar.close()

    accelerator.wait_for_everyone()

    # Generate sample images for visual inspection
    # if accelerator.is_main_process:
    if epoch % args.save_images_epochs == 0 or epoch == args.num_epochs - 1:
        pipeline = DDPMConditionalPipeline(
            unet=accelerator.unwrap_model(ema_model.averaged_model if args.use_ema else model),
            scheduler=noise_scheduler,
        )

        generator = torch.manual_seed(0)
        # run pipeline in inference (sample random noise and denoise)
        attr = torch.tensor([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]).long().repeat(eval_batch_size_,1)
        y = (lin(emb(attr)))
        images = pipeline(generator=generator, batch_size=args.eval_batch_size, output_type="numpy", hidden_states=y.to("cuda"))["sample"]

        # denormalize the images and save to tensorboard
        images_processed = (images * 255).round().astype("uint8")
        imgs_t = images_processed.transpose(0, 3, 1, 2)
        imgs_plt = images_processed.transpose(0, 1, 2, 3)

        fig = plt.figure(figsize=(8, 8))
        columns = 1
        rows = 1
        for i in range(1, columns*rows +1):
            img = imgs_plt[i-1]
            fig.add_subplot(rows, columns, i)
            plt.imshow(img)
        plt.show()

    if epoch % args.save_model_epochs == 0 or epoch == args.num_epochs - 1:
        # save the model
        # if args.push_to_hub:
        push_to_hub(args, pipeline, repo, commit_message=f"Epoch {epoch}", blocking=False)
        # else:
        pipeline.save_pretrained(args.output_dir)

    accelerator.wait_for_everyone()

accelerator.end_training()

  0%|          | 0/10174 [00:00<?, ?it/s]

### Misc

In [ ]:
# del ddpm
import torch
import gc
gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()
# torch.cuda.memory_summary(device=None, abbreviated=False)
!nvidia-smi

In [ ]:
m = nn.Linear(40, 30)
input = torch.randn(128, 40)
output = (m(input))
print(output.size())

In [ ]:
print(input)
print(output)

In [ ]:
input = torch.randn(128,40).long()
input

In [ ]:
embedding = nn.Embedding(128*40, 128*40)
embedding(input)

In [ ]:
embedding = nn.Embedding(2, 40, padding_idx=0)
input = torch.LongTensor([[ 0,  0,  0,  0,0,  0,  0,  0,0,0],
        [ 0,  0,  0, 1,0,  0,  0,  0,0,0],
        [ 0,  0,  0, 0,0,  0,  0,  0,0,0],
        [ 0,  0,  1, 1,0,  0,  0,  0,0,0]])
print(input)
e = embedding(input)
e.shape

In [ ]:
attr = torch.tensor([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]).long()
attr.unsqueeze(0)

In [ ]:
attr.repeat(32,1).shape

In [ ]:
attr = torch.tensor([[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]).long()

In [ ]:
attr.shape

In [ ]:
c = torch.cat([attr,attr])
c.shape

In [ ]:
cc = torch.cat([attr] * 2)
cc.shape